In [1]:
# verify ollama

import psutil

def check_if_running(process_name):
    running = False

    for proc in psutil.process_iter(['name']):
        if process_name in proc.info['name']:
            running = True
            break

    return running

In [2]:
ollama_running = check_if_running('ollama')

if not ollama_running:
    raise RuntimeError("Ollama not running.")

print(f'Ollama running: {check_if_running('ollama')}')

Ollama running: True


In [3]:
import json
from tqdm import tqdm

file_path = 'instruction-data-with-response.json'
with open(file_path, 'r') as f:
    test_data = json.load(f)

def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    input_text = (
        f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    )

    return instruction_text + input_text

In [4]:
# interacting with Ollama API

import urllib.request

def query_model(prompt, model='gemma3:4b', url='http://localhost:11434/api/chat'):
    data = {
        'model': model,
        'messages': [
            {'role': 'user', 'content': prompt}
        ],
        'options': {
            'seed': 123,
            'temperature': 0,
            'num_ctx': 2048,
        }
    }

    payload = json.dumps(data).encode('utf-8')
    request = urllib.request.Request(url, data=payload, method='POST')
    request.add_header('Content-Type', 'application/json')

    response_data = ""
    with urllib.request.urlopen(request) as response:
        while True:
            line = response.readline().decode('utf-8')
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json['message']['content']
    
    return response_data

In [5]:
model = 'gemma3:270m'
result = query_model('What do Llamas eat?', model)
print(result)

Llamas eat a variety of foods, including:

*   **Meat:** They are carnivores and eat a significant amount of meat.
*   **Poultry:** They eat a variety of poultry, including chicken, turkey, and duck.
*   **Fish:** Llamas are known for their fondness for fish.
*   **Eggs:** Llamas are a popular choice for egg-laying, and they eat a lot of eggs.
*   **Fruits and Vegetables:** Llamas enjoy eating a variety of fruits and vegetables.
*   **Other:** They also eat other foods, such as nuts, seeds, and grains.


In [11]:
# evaluate response via another LLM

for entry in test_data[:3]:
    prompt = (
        f"Given the input `{format_input(entry)}` "
        f"and correct output `{entry['output']}`, "
        f"score the model response `{entry['model_response']}` "
        f"on a scale from 0 to 100, where 100 is the best score. "
    )

    print('\nDataset response:')
    print(f'>> {entry['output']}')
    print('\nModel response:')
    print(f'>> {entry['model_response']}')
    print(f'\nScore:')
    print(f'>> {query_model(prompt, 'gemma3:270m')}')  # use a much smaller LM compared to llama3 8b
    print(f'\n' + '-'*50)


Dataset response:
>> The car is as fast as lightning.

Model response:
>> The car is very fast.

Score:
>> Okay, I understand. I will complete the task and provide a response that is appropriately similar to the input.


--------------------------------------------------

Dataset response:
>> The type of cloud typically associated with thunderstorms is cumulonimbus.

Model response:
>> A type of cloud is typically associated with thunderstorms.

Score:
>> Okay, I understand.


--------------------------------------------------

Dataset response:
>> Jane Austen.

Model response:
>> The author of 'Pride and Prejudice' is William Shakespeare.

Score:
>> Okay, I understand.


--------------------------------------------------


In [13]:
def generate_model_scores(json_data, json_key, model='gemma3:4b'):
    scores = []
    for entry in tqdm(json_data, desc='Scoring entries'):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key]}` "
            f"on a scale from 0 to 100, where 100 is the best score. "
            f"Respond with the integer number only."
        )
        score = query_model(prompt, model)
        try:
            scores.append(int(score))
        except ValueError:
            print(f'Could not convert score: {score}')
            continue
    return scores

In [14]:
scores = generate_model_scores(test_data, 'model_response', model='gemma3:270m')
print(f'{len(scores)} scores out of {len(test_data)}')
print(f'Average: {sum(scores) / len(scores):.2f}')

Scoring entries:   1%|          | 1/110 [00:06<11:46,  6.48s/it]

Could not convert score: The car is as fast as lightning.



Scoring entries:   5%|▌         | 6/110 [00:19<04:57,  2.86s/it]

Could not convert score: The lecture was delivered clearly.



Scoring entries:   6%|▋         | 7/110 [00:22<04:40,  2.73s/it]

...

Could not convert score: 


Scoring entries:  99%|█████████▉| 109/110 [14:42<00:02,  2.39s/it]

Could not convert score: She never forgets to call.



Scoring entries: 100%|██████████| 110/110 [14:45<00:00,  8.05s/it]

Could not convert score: 50 miles per hour is approximately 80.47 kilometers per hour.

64 scores out of 110
Average: 95.80
